In [60]:
import pandas as pd
%matplotlib inline

import boto3
from io import StringIO

In [61]:
# df=pd.read_csv("ml_log.csv",parse_dates=True)#infer_datetime_format=True) ,index_col=['time','ticker']

In [62]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="model-support-files"
Key="17_ml_log.csv"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df = pd.read_csv(read_file['Body'],sep=',',index_col=['datetime'],parse_dates=True)

In [90]:
df=df.loc[:, df.columns != 'ATR']
df.head()

,security,close,RSI,STOCHASTIC
datetime,,,,
2010-12-30 22:00:00,EUR_USD,1.32855,NaN,NaN
2010-12-31 22:00:00,EUR_USD,1.33815,NaN,NaN
2011-01-01 22:00:00,EUR_USD,1.33280,NaN,NaN
2011-01-02 22:00:00,EUR_USD,1.32512,NaN,NaN
2011-01-03 22:00:00,EUR_USD,1.32919,NaN,NaN


In [91]:
# df['datetime']=pd.to_datetime(df['datetime'])
df['fwd_returns']=df.groupby("security")["close"].pct_change(5)

In [92]:
df.sort_values(by='datetime',inplace=True)

In [93]:
df=df.reset_index().drop(columns=['datetime','security','close'])

In [94]:
df.head(100)

,RSI,STOCHASTIC,fwd_returns
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
95,60.698443,68.297987,0.010432
96,61.555018,63.554618,0.000618
97,62.765973,59.444566,0.005820
98,66.785263,72.651911,0.019728


In [96]:
df.dropna().to_csv('ml_log_processed.csv',index=False)

In [95]:
df.dropna().to_csv(("processed_"+Key),index=False)

### Old Format

In [2]:
# df=pd.read_csv("ml_log.csv",index_col=['time','ticker'],parse_dates=True)
df=pd.read_csv("ml_log.csv",index_col=['time'],parse_dates=True)

In [3]:
df.head()

,ticker,close,sma1,sma2
time,,,,
2019-01-28 22:00:00,EUR_USD,1.14113,1.135365,1.137738
2019-01-28 22:00:00,GBP_USD,1.30556,1.294032,1.280558
2019-01-29 22:00:00,EUR_USD,1.14063,1.135649,1.138145
2019-01-29 22:00:00,GBP_USD,1.30543,1.296327,1.282924
2019-01-30 22:00:00,EUR_USD,1.14360,1.136307,1.138780


In [16]:
df['fwd_returns']=df.groupby("ticker")["close"].pct_change(1)

In [17]:
df=df.reset_index().drop(columns=['time','ticker','close'])

In [18]:
df.dropna().to_csv("ml_log_processed.csv",index=False)

In [19]:
df.head()

,sma1,sma2,fwd_returns
0,1.135365,1.137738,NaN
1,1.294032,1.280558,NaN
2,1.135649,1.138145,-0.000438
3,1.296327,1.282924,-0.000100
4,1.136307,1.138780,0.002604


## QGRID

In [51]:
# ! pip install qgrid

# ! jupyter nbextension enable --py --sys-prefix qgrid
# ! jupyter nbextension enable --py --sys-prefix widgetsnbextension

# import qgrid
# qgrid_widget=qgrid.show_grid(df,show_toolbar=True)
# qgrid_widget